In [1]:
# On import les packages
from dash import Dash, html, dash_table, dcc
import pandas as pd
import plotly.express as px
import numpy as np
import dash_daq as daq
import dash_bootstrap_components as dbc


# On lit le jeu de données des décès en France ligne par ligne
lines = []
with open("data/deces-2023-m08.txt", "r", encoding="utf-8") as file:
    lines.extend(file.readlines())
with open("data/deces-2022-t4.txt", "r", ) as file:
    lines.extend(file.readlines())
# On charge le jeu de données pour pouvoir connaître la position géographique des villes en France
positions_geo = pd.read_csv('data/communes-departement-region.csv', usecols=[0, 1, 5, 6])


# On lit le jeu de données des positions des pays étrangers ligne par ligne
with open("data/positions-pays-étrangers.txt", "r") as file2:
    country_data = file2.readlines()

# On ajoute un 0 dans le cas où les codes on seulement 4 chiffres
positions_geo['code_commune_INSEE'] = positions_geo['code_commune_INSEE'].str.zfill(5)

# Ouvrir jeu de données esperance vie
lines_esperance =[]
with open("data/esperance_vie.txt", "r") as fileEsperance:   
    lines_esperance = fileEsperance.readlines()

# On initialise les listes vides pour chaque colonne du tableau
names = []
gender = []
dob = []
birthplace_code_list = []
birthplace = []
birthplace_details = []
year_list = []
dod = []
deathplace_code = []
death_cert_number = []
noms = []
prenoms = []
Age = []
same_places = []
deathplace = []

# Créez un dictionnaire pour stocker les correspondances deathplace_code_text -> birthplace_text
deathplace_mapping = {}

# Boucle pour extraire les données et trouver les correspondances
for line in lines:
    name = line[0:80].strip()
    nom, prenom = name.split('*')
    prenom = prenom.strip('/')
    gender_code = line[80:81]
    date_year = line[81:85]
    date_month = line[85:87]
    date_day = line[87:89]
    birthplace_code = line[89:94].strip()
    birthplace_text = line[94:124].strip()
    birthplace_details_text = line[124:154].strip()
    date_of_death_year = line[154:158]
    date_of_death_month = line[158:160]
    date_of_death_day = line[160:162]
    deathplace_code_text = line[162:167].strip()
    death_cert_number_text = line[167:176].strip()
    age = int(date_of_death_year) - int(date_year)

    # On doit convertir le code du genre en string
    if gender_code == '1':
        gender_text = "Masculin"
    elif gender_code == '2':
        gender_text = "Féminin"
    else:
        gender_text = "Inconnu"

    if birthplace_details_text == '':
        birthplace_details_text = "FRANCE"

    if deathplace_code_text == birthplace_code:
        new_place = (deathplace_code_text, birthplace_text)
        if new_place not in same_places:
            same_places.append(new_place)

        # Stockez la correspondance dans le dictionnaire
        deathplace_mapping[deathplace_code_text] = birthplace_text

    names.append(name)
    gender.append(gender_text)
    year_list.append(date_year)
    dob.append(date_day + "/" + date_month + "/" + date_year)
    birthplace_code_list.append(birthplace_code)
    birthplace.append(birthplace_text)
    birthplace_details.append(birthplace_details_text)
    dod.append(date_of_death_day + "/" + date_of_death_month + "/" + date_of_death_year)
    deathplace_code.append(deathplace_code_text)
    death_cert_number.append(death_cert_number_text)
    noms.append(nom)
    prenoms.append(prenom)
    Age.append(age)

# Boucle principale pour le traitement des lignes
for line in lines:
    deathplace_code_text = line[162:167].strip()
    deathplace_value = deathplace_mapping.get(deathplace_code_text, "NULL")
    deathplace.append(deathplace_value)

# Créez le tableau à partir des listes créées
data = {
    "Nom": noms,
    "Prenom(s)": prenoms,
    "Sex": gender,
    "Year": year_list,
    "Date of Birth": dob,
    "Age": Age,
    "Birthplace Code": birthplace_code_list,
    "Birthplace": birthplace,
    "Birthplace Details": birthplace_details,
    "Date of Death": dod,
    "Deathplace Code": deathplace_code,
    "Death Certificate Number": death_cert_number,
    "Death Place": deathplace,
}

# Créez le DataFrame avec pandas
df = pd.DataFrame(data)


# On crée un DataFrame avec toute l'information ensemble pour pouvoir comparer les Villes avec leur position géographique
positions_geo.drop_duplicates(inplace=True)
merged_df_death = df.merge(positions_geo, left_on='Deathplace Code', right_on='code_commune_INSEE', how='left')
merged_df_death = merged_df_death.rename(columns={'longitude': 'longitude_death', 'latitude': 'latitude_death'})


merged_df_birth = df.merge(positions_geo, left_on='Birthplace Code', right_on='code_commune_INSEE', how='left')
merged_df_birth = merged_df_birth.rename(columns={'longitude': 'longitude_birth', 'latitude': 'latitude_birth'})

# Fusionnez les données de décès avec les données de naissance en utilisant la colonne "Nom" comme clé
merged_df = merged_df_death.merge(merged_df_birth[['longitude_birth', 'latitude_birth']], left_index=True, right_index=True, how='left')
merged_df.loc[merged_df['Birthplace Details'] != 'FRANCE', ['latitude_birth', 'longitude_birth']] = np.nan

country_coordinates = {}

for line in country_data:
    parts = line.strip().split()
    if len(parts) == 3:
        country = parts[0].replace('*', ' ')
        latitude, longitude = parts[1], parts[2]
        country_coordinates[country] = (latitude, longitude)

def update_coordinates(row):
    country = row['Birthplace Details']
    if country in country_coordinates:
        latitude, longitude = country_coordinates[country]
        row['latitude_birth'] = latitude
        row['longitude_birth'] = longitude
    return row

merged_df = merged_df.apply(update_coordinates, axis=1)
merged_df['density'] = 1

years_esperance_list = []
esperances_hommes_list = []
esperances_femmes_list = []
mortalites_infantiles_list = []

for line in lines_esperance:
    year_esperance = line[0:4]
    esperance_femmes = line[5:9]
    esperance_hommes = line[10:14]

    years_esperance_list.append(year_esperance)
    esperances_hommes_list.append(esperance_hommes)
    esperances_femmes_list.append(esperance_femmes)
# Créez le tableau à partir des listes créées
data_esp = {
    "Year": years_esperance_list,
    "Esperance Hommes": esperances_hommes_list,
    "Esperance Femmes": esperances_femmes_list,
}

# Créez le DataFrame avec pandas
df_esp = pd.DataFrame(data_esp)

# Initialisez l'application Dash
app = Dash(__name__, suppress_callback_exceptions=True, assets_folder='../assets', external_stylesheets=[dbc.themes.DARKLY])

theme = {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


In [13]:
df_esp

,Year,Esperance Hommes,Esperance Femmes
0,1950,"63,4","69,2"
1,1951,"63,1","68,9"
2,1952,"64,4","70,2"
3,1953,"64,3","70,3"
4,1954,"65,0","71,2"
...,...,...,...
65,2015,"79,0","85,1"
66,2016,"79,3","85,3"
67,2017,"79,5","85,4"
68,2018,"79,6","85,5"


In [2]:
positions_geo[positions_geo.duplicated('code_commune_INSEE')]

,code_commune_INSEE,nom_commune_postal,latitude,longitude


In [3]:
merged_df_birth = df.merge(positions_geo, left_on='Birthplace Code', right_on='code_commune_INSEE', how='left')
merged_df_birth = merged_df_birth.rename(columns={'longitude': 'longitude_birth', 'latitude': 'latitude_birth'})
grouped_df_birth = merged_df_birth.groupby('Nom').agg({
    'longitude_birth': 'first',  # Utilisez 'first' pour obtenir le premier valeur non nulle.
    'latitude_birth': 'first',   # Utilisez 'first' pour obtenir le premier valeur non nulle.
}).reset_index()



In [4]:
grouped_df_birth

,Nom,longitude_birth,latitude_birth
0,,NaN,NaN
1,AALHADAAY,NaN,NaN
2,AARAB,NaN,NaN
3,AARRASS,NaN,NaN
4,AARROUD,NaN,NaN
...,...,...,...
32261,ZUTTERLING,6.448560,49.187623
32262,ZWENGER,NaN,NaN
32263,ZWISLER,6.847342,43.457463
32264,ZYCHOWICZ,2.913232,49.778445


In [5]:
merged_df_birth.head()

,Nom,Prenom(s),Sex,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_birth,longitude_birth
0,HYVERNAT,CHRISTOPHE,Masculin,07/04/1970,53,71270,MACON,FRANCE,24/08/2023,01004,175,AMBERIEU-EN-BUGEY,71270,MACON,46.320551,4.818425
1,PLENARD,ROLANDE JOSEPHINE,Féminin,14/05/1932,91,01365,SAINT-JEAN-SUR-VEYLE,FRANCE,04/08/2023,01004,160,AMBERIEU-EN-BUGEY,01365,ST JEAN SUR VEYLE,46.268363,4.921984
2,AGUETTAZ,ODETTE PAULE,Féminin,27/04/1918,105,01454,VIRIGNIN,FRANCE,17/08/2023,01004,165,AMBERIEU-EN-BUGEY,01454,VIRIGNIN,45.726739,5.712823
3,MELLOUL,SIMONE,Féminin,07/02/1936,87,99350,RABAT,MAROC,24/08/2023,01004,174,AMBERIEU-EN-BUGEY,NaN,NaN,NaN,NaN
4,MARECHAL,FRANCK TONY,Masculin,02/06/1971,52,01053,BOURG-EN-BRESSE,FRANCE,09/08/2023,01004,162,AMBERIEU-EN-BUGEY,01053,BOURG EN BRESSE,46.205152,5.246021


In [6]:


# Fusionnez les données de décès avec les données de naissance en utilisant la colonne "Nom" comme clé
merged_df = merged_df_death.merge(grouped_df_birth[['Nom', 'longitude_birth', 'latitude_birth']], on='Nom', how='left')

In [7]:
merged_df.head()

,Nom,Prenom(s),Sex,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_death,longitude_death,longitude_birth,latitude_birth
0,HYVERNAT,CHRISTOPHE,Masculin,07/04/1970,53,71270,MACON,FRANCE,24/08/2023,01004,175,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,4.818425,46.320551
1,PLENARD,ROLANDE JOSEPHINE,Féminin,14/05/1932,91,01365,SAINT-JEAN-SUR-VEYLE,FRANCE,04/08/2023,01004,160,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,4.921984,46.268363
2,AGUETTAZ,ODETTE PAULE,Féminin,27/04/1918,105,01454,VIRIGNIN,FRANCE,17/08/2023,01004,165,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,5.712823,45.726739
3,MELLOUL,SIMONE,Féminin,07/02/1936,87,99350,RABAT,MAROC,24/08/2023,01004,174,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,NaN,NaN
4,MARECHAL,FRANCK TONY,Masculin,02/06/1971,52,01053,BOURG-EN-BRESSE,FRANCE,09/08/2023,01004,162,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,5.246021,46.205152


In [8]:
merged_df.loc[merged_df['Birthplace Details'] != 'FRANCE', ['latitude_birth', 'longitude_birth']] = np.nan

In [2]:
international_countries = merged_df[merged_df['Birthplace Details'] != 'FRANCE']

In [3]:
international_countries

,Nom,Prenom(s),Sex,Year,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_death,longitude_death,longitude_birth,latitude_birth,density
3,CAZORLA,JOSEPHINE,Féminin,1931,23/11/1931,92,99352,"BENI-SAF, ORAN",ALGERIE,10/07/2023,01004,148,AMBERIEU-EN-BUGEY,01004,AMBERIEU EN BUGEY,45.960848,5.372926,3.0588,36.7538,1
15,ROS,SYPHAN,Féminin,1954,18/02/1954,69,99234,KOMPONG SOM,CAMBODGE,13/07/2023,01014,20,NULL,01014,ARBENT,46.283494,5.690617,104.9282,11.5564,1
20,DHERIN,MARIA EUGENIA VENERANDA,Féminin,1926,17/02/1926,97,99127,MONTJOVET,ITALIE,13/07/2023,01033,68,VALSERHONE,01033,BELLEGARDE SUR VALSERINE,46.106790,5.832027,12.4964,41.9028,1
26,KHALAF,GABY,Masculin,1943,27/06/1943,80,99206,ALEP,SYRIE,24/07/2023,01034,190,BELLEY,01034,BELLEY,45.749470,5.684124,36.2765,33.5138,1
63,ELIAS CORDEIRO,RUI MANUEL,Masculin,1957,05/05/1957,66,99139,BARREIRO-SETUBAL,PORTUGAL,20/07/2023,01053,403,BOURG-EN-BRESSE,01053,BOURG EN BRESSE,46.205152,5.246021,-9.1393,38.7223,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329477,BETARI,YOUNES,Masculin,1980,05/05/1980,42,99350,OUJDA,MAROC,08/10/2022,99134,2523,FERNAN NUNEZ,NaN,NaN,NaN,NaN,-7.0926,31.7917,1
329482,CHENIOUR,HABIB,Masculin,1960,07/01/1960,62,99351,TUNIS,TUNISIE,18/09/2022,99351,438,TUNIS,NaN,NaN,NaN,NaN,10.1815,36.8065,1
329483,HAGGIAG,RAYMONDE MESSAOUDA,Féminin,1930,02/06/1930,92,99351,TUNIS,TUNISIE,05/12/2022,99351,439,TUNIS,NaN,NaN,NaN,NaN,10.1815,36.8065,1
329484,BEDJGUELAL,AHMED,Masculin,1935,21/07/1935,87,99352,ACHELOUF,ALGERIE,26/11/2022,99350,138,CASABLANCA,NaN,NaN,NaN,NaN,3.0588,36.7538,1


In [4]:
international_countries['Birthplace Details'].unique()

array(['ALGERIE', 'CAMBODGE', 'ITALIE', 'SYRIE', 'PORTUGAL', 'MAROC',
       'ESPAGNE', 'TURQUIE', 'SUEDE', 'ETATS-UNIS', 'SUISSE',
       'LA REUNION', 'TUNISIE', 'SERBIE',
       'CONGO (REPUBLIQUE DEMOCRATIQUE', 'CAMEROUN', 'ALLEMAGNE',
       'MARTINIQUE', 'POLOGNE', "COTE D'IVOIRE", 'BELGIQUE', 'VIET NAM',
       'GUADELOUPE', 'ROYAUME-UNI', 'PAYS-BAS', 'ROUMANIE', 'DANEMARK',
       'AUTRICHE', 'SLOVAQUIE', 'SENEGAL', 'SRI LANKA', 'EGYPTE',
       'ISRAEL', 'GRECE', 'MONACO', 'COMORES', 'RUSSIE', 'CANADA', 'LAOS',
       'MAURICE', 'FINLANDE', 'LIBAN', 'AUSTRALIE', 'IRLANDE, OU EIRE',
       'LETTONIE', 'TOGO', 'PEROU', "PROVINCES ESPAGNOLES D'AFRIQUE",
       'UKRAINE', 'INDE', 'BULGARIE', 'PAYS INCONNU', 'ARMENIE', 'CONGO',
       'ARGENTINE', 'IRAN', 'MADAGASCAR', 'GUYANE', 'BRESIL', 'SURINAME',
       'HONGRIE', 'LUXEMBOURG', 'COREE', 'THAILANDE', 'BURKINA', 'MALI',
       'COLOMBIE', 'TANGER', 'GHANA', 'GEORGIE', 'CHINE',
       'VIET NAM DU NORD', 'CROATIE', 'NORVEGE', 'VIE

In [30]:
def remove_quotes(text):
    return text.strip('"')

country_coordinates = {}

for line in country_data:
    parts = line.strip().split()
    if len(parts) == 3:
        country = remove_quotes(parts[0])
        latitude, longitude = parts[1], parts[2]
        country_coordinates[country] = (latitude, longitude)

def update_coordinates(row):
    country = row['Birthplace Details']
    if country in country_coordinates:
        latitude, longitude = country_coordinates[country]
        row['latitude_birth'] = latitude
        row['longitude_birth'] = longitude
    return row

merged_df = merged_df.apply(update_coordinates, axis=1)

In [35]:
merged_df[merged_df['Birthplace Details'] == 'CAMEROUN ET TOGO'].head()

,Nom,Prenom(s),Sex,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_death,longitude_death,longitude_birth,latitude_birth
30542,CHARAMON,CATHERINE ROBERTE,Féminin,20/03/1948,75,99305,BATOURI,CAMEROUN ET TOGO,22/08/2023,34172,2777,MONTPELLIER,34172,MONTPELLIER,43.613441,3.868517,NaN,NaN
30543,CHARAMON,CATHERINE ROBERTE,Féminin,20/03/1948,75,99305,BATOURI,CAMEROUN ET TOGO,22/08/2023,34172,2777,MONTPELLIER,34172,MONTPELLIER,43.613441,3.868517,NaN,NaN
30544,CHARAMON,CATHERINE ROBERTE,Féminin,20/03/1948,75,99305,BATOURI,CAMEROUN ET TOGO,22/08/2023,34172,2777,MONTPELLIER,34172,MONTPELLIER,43.613441,3.868517,NaN,NaN
30545,CHARAMON,CATHERINE ROBERTE,Féminin,20/03/1948,75,99305,BATOURI,CAMEROUN ET TOGO,22/08/2023,34172,2777,MONTPELLIER,34172,MONTPELLIER,43.613441,3.868517,NaN,NaN
67285,MBEDI PRISO,EBENEZER,Masculin,13/09/1939,84,99305,MESSA YAOUNDE,CAMEROUN ET TOGO,20/08/2023,75119,678,PARIS 19E ARRONDISSEMENT,75119,PARIS 19,48.886869,2.384694,NaN,NaN


In [32]:
merged_df[merged_df['Birthplace Details'] == 'ACORES, MADERE']

,Nom,Prenom(s),Sex,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_death,longitude_death,longitude_birth,latitude_birth
65193,DE PAULO CANHA,MARIA INES,Féminin,10/01/1960,63,99319,"PONTA DO SOL, COMMUNE DE PONTA","ACORES, MADERE",16/08/2023,74012,190,NULL,74012,ANNEMASSE,46.190973,6.242507,NaN,NaN
66979,MORAIS,AIDA DE JESUS,Féminin,27/03/1945,78,99319,"NOSSA SENHORA DA LUZ, SAO VICE","ACORES, MADERE",05/08/2023,75115,2394,PARIS 15E ARRONDISSEMENT,75115,PARIS 15,48.840155,2.293559,NaN,NaN
69305,FERREIRA,ANTONIO,Masculin,08/03/1944,79,99319,LAGE COMMUNE DE VILA VERDE,"ACORES, MADERE",30/07/2023,77048,43,NULL,77048,BOURRON MARLOTTE,48.335261,2.701700,NaN,NaN
69933,GOMES,DOMINGOS,Masculin,30/06/1939,84,99319,SANTA CATARINA,"ACORES, MADERE",25/07/2023,77390,91,NULL,77390,ROISSY EN BRIE,48.790086,2.661224,NaN,NaN


In [28]:
country_coordinates = {}
for line in country_data:
    parts = line.strip().split()
    if len(parts) == 3:
        country, latitude, longitude = parts
        country_coordinates[country] = (latitude, longitude)

In [11]:
merged_df[merged_df['Birthplace'] == 'BORDEAUX']

,Nom,Prenom(s),Sex,Date of Birth,Age,Birthplace Code,Birthplace,Birthplace Details,Date of Death,Deathplace Code,Death Certificate Number,Death Place,code_commune_INSEE,nom_commune_postal,latitude_death,longitude_death,longitude_birth,latitude_birth
1336,JEANSON,TRISTAN JOSEPH MICHEL,Masculin,13/07/2020,3,33063,BORDEAUX,FRANCE,14/08/2023,04070,246,DIGNE-LES-BAINS,04070,DIGNE LES BAINS,44.090872,6.235903,-0.573697,44.857245
1337,JEANSON,TRISTAN JOSEPH MICHEL,Masculin,13/07/2020,3,33063,BORDEAUX,FRANCE,14/08/2023,04070,246,DIGNE-LES-BAINS,04070,DIGNE LES BAINS,44.090872,6.235903,-0.573697,44.857245
2241,SABLIER,MONIQUE PAULETTE,Féminin,30/04/1943,80,33063,BORDEAUX,FRANCE,09/08/2023,06029,891,CANNES,06029,CANNES,43.552620,7.004276,-0.573697,44.857245
2242,SABLIER,MONIQUE PAULETTE,Féminin,30/04/1943,80,33063,BORDEAUX,FRANCE,09/08/2023,06029,891,CANNES,06029,CANNES,43.552620,7.004276,-0.573697,44.857245
5881,BARBE,FABIENNE,Féminin,21/03/1960,63,33063,BORDEAUX,FRANCE,25/07/2023,07276,2,NULL,07276,ST MICHEL D AURANCE,44.898718,4.456759,4.888458,45.770770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89304,DELORT,MARCELLE JEANNE,Féminin,27/05/1940,83,33063,BORDEAUX,FRANCE,03/07/2023,98724,26,HUAHINE,98724,HUAHINE,NaN,NaN,-0.573697,44.857245
89305,DELORT,MARCELLE JEANNE,Féminin,27/05/1940,83,33063,BORDEAUX,FRANCE,03/07/2023,98724,26,HUAHINE,98724,HUAHINE,NaN,NaN,-0.573697,44.857245
89306,DELORT,MARCELLE JEANNE,Féminin,27/05/1940,83,33063,BORDEAUX,FRANCE,03/07/2023,98724,26,HUAHINE,98724,HUAHINE,NaN,NaN,-0.573697,44.857245
89307,DELORT,MARCELLE JEANNE,Féminin,27/05/1940,83,33063,BORDEAUX,FRANCE,03/07/2023,98724,26,HUAHINE,98724,HUAHINE,NaN,NaN,-0.573697,44.857245


In [36]:
country_coordinates

{'MAROC': ('31.7917', '-7.0926'),
 'PORTUGAL': ('38.7223', '-9.1393'),
 'SERBIE': ('44.7872', '20.4573'),
 'ALGERIE': ('36.7538', '3.0588'),
 'MARTINIQUE': ('14.6415', '-61.0242'),
 'AUTRICHE': ('48.2082', '16.3738'),
 'MALI': ('12.6392', '-8.0029'),
 'PAYS-BAS': ('52.3676', '4.9041'),
 'TOGO': ('6.1228', '1.2275'),
 'SUISSE': ('46.8182', '8.2275'),
 'RUSSIE': ('55.7558', '37.6176'),
 'ESPAGNE': ('40.4168', '-3.7038'),
 'CAMBODGE': ('11.5564', '104.9282'),
 'ITALIE': ('41.9028', '12.4964'),
 'TURQUIE': ('41.0082', '28.9784'),
 'MAURICE': ('-20.3484', '57.5522'),
 'HONGRIE': ('47.4979', '19.0402'),
 'BRESIL': ('-15.7801', '-47.9292'),
 'POLOGNE': ('52.2297', '21.0122'),
 'TUNISIE': ('36.8065', '10.1815'),
 'BELGIQUE': ('50.8503', '4.3517'),
 'ROYAUME-UNI': ('51.5072', '-0.1276'),
 'ALLEMAGNE': ('52.5200', '13.4050'),
 'GUADELOUPE': ('16.2650', '-61.5500'),
 'NOUVELLE-ZELANDE': ('-41.2865', '174.7762'),
 'GUYANE': ('3.9339', '-53.1258'),
 'LIBAN': ('33.8886', '35.4955'),
 'GRECE': ('37.9

In [13]:
# On import les packages
from dash import Dash, html, dash_table, dcc
import pandas as pd
import plotly.express as px
import numpy as np

# On lit le jeu de données des décès en France ligne par ligne
lines = []
with open("./deces-2023-m08.txt", "r") as file:
    lines = file.readlines()

# On charge le jeu de données pour pouvoir connaître la position géographique des villes en France
positions_geo = pd.read_csv('communes-departement-region.csv', usecols=[0, 1, 5, 6])


# On lit le jeu de données des positions des pays étrangers ligne par ligne
with open("./positions-pays-étrangers.txt", "r") as file2:
    country_data = file2.readlines()

# On ajoute un 0 dans le cas où les codes on seulement 4 chiffres
positions_geo['code_commune_INSEE'] = positions_geo['code_commune_INSEE'].str.zfill(5)


# On initialise les listes vides pour chaque colonne du tableau
names = []
gender = []
dob = []
birthplace_code_list = []
birthplace = []
birthplace_details = []
dod = []
deathplace_code = []
death_cert_number = []
noms = []
prenoms = []
Age = []
same_places = []
deathplace = []

# Créez un dictionnaire pour stocker les correspondances deathplace_code_text -> birthplace_text
deathplace_mapping = {}

# Boucle pour extraire les données et trouver les correspondances
for line in lines:
    name = line[0:80].strip()
    nom, prenom = name.split('*')
    prenom = prenom.strip('/')
    gender_code = line[80:81]
    date_year = line[81:85]
    date_month = line[85:87]
    date_day = line[87:89]
    birthplace_code = line[89:94].strip()
    birthplace_text = line[94:124].strip()
    birthplace_details_text = line[124:154].strip()
    date_of_death_year = line[154:158]
    date_of_death_month = line[158:160]
    date_of_death_day = line[160:162]
    deathplace_code_text = line[162:167].strip()
    death_cert_number_text = line[167:176].strip()
    age = int(date_of_death_year) - int(date_year)

    # On doit convertir le code du genre en string
    if gender_code == '1':
        gender_text = "Masculin"
    elif gender_code == '2':
        gender_text = "Féminin"
    else:
        gender_text = "Inconnu"

    if birthplace_details_text == '':
        birthplace_details_text = "FRANCE"

    if deathplace_code_text == birthplace_code:
        new_place = (deathplace_code_text, birthplace_text)
        if new_place not in same_places:
            same_places.append(new_place)

        # Stockez la correspondance dans le dictionnaire
        deathplace_mapping[deathplace_code_text] = birthplace_text

    names.append(name)
    gender.append(gender_text)
    dob.append(date_day + "/" + date_month + "/" + date_year)
    birthplace_code_list.append(birthplace_code)
    birthplace.append(birthplace_text)
    birthplace_details.append(birthplace_details_text)
    dod.append(date_of_death_day + "/" + date_of_death_month + "/" + date_of_death_year)
    deathplace_code.append(deathplace_code_text)
    death_cert_number.append(death_cert_number_text)
    noms.append(nom)
    prenoms.append(prenom)
    Age.append(age)

# Boucle principale pour le traitement des lignes
for line in lines:
    deathplace_code_text = line[162:167].strip()
    deathplace_value = deathplace_mapping.get(deathplace_code_text, "NULL")
    deathplace.append(deathplace_value)

# Créez le tableau à partir des listes créées
data = {
    "Nom": noms,
    "Prenom(s)": prenoms,
    "Sex": gender,
    "Date of Birth": dob,
    "Age": Age,
    "Birthplace Code": birthplace_code_list,
    "Birthplace": birthplace,
    "Birthplace Details": birthplace_details,
    "Date of Death": dod,
    "Deathplace Code": deathplace_code,
    "Death Certificate Number": death_cert_number,
    "Death Place": deathplace,
}

# Créez le DataFrame avec pandas
df = pd.DataFrame(data)


# On crée un DataFrame avec toute l'information ensemble pour pouvoir comparer les Villes avec leur position géographique
# Realizar la fusión para las coordenadas de muerte
positions_geo.drop_duplicates(inplace=True)
merged_df_death = df.merge(positions_geo, left_on='Deathplace Code', right_on='code_commune_INSEE', how='left')
merged_df_death = merged_df_death.rename(columns={'longitude': 'longitude_death', 'latitude': 'latitude_death'})


merged_df_birth = df.merge(positions_geo, left_on='Birthplace Code', right_on='code_commune_INSEE', how='left')
merged_df_birth = merged_df_birth.rename(columns={'longitude': 'longitude_birth', 'latitude': 'latitude_birth'})

grouped_df_birth = merged_df_birth.groupby('Nom').agg({
    'longitude_birth': 'first',  # Utilisez 'first' pour obtenir le premier valeur non nulle.
    'latitude_birth': 'first',   # Utilisez 'first' pour obtenir le premier valeur non nulle.
}).reset_index()


# Fusionnez les données de décès avec les données de naissance en utilisant la colonne "Nom" comme clé
merged_df = merged_df_death.merge(grouped_df_birth[['Nom', 'longitude_birth', 'latitude_birth']], on='Nom', how='left')
merged_df.loc[merged_df['Birthplace Details'] != 'FRANCE', ['latitude_birth', 'longitude_birth']] = np.nan

country_coordinates = {}

for line in country_data:
    parts = line.strip().split()
    if len(parts) == 3:
        country = parts[0].replace('*', ' ')
        latitude, longitude = parts[1], parts[2]
        country_coordinates[country] = (latitude, longitude)

    
def update_coordinates(row):
    country = row['Birthplace Details']
    if country in country_coordinates:
        latitude, longitude = country_coordinates[country]
        row['latitude_birth'] = latitude
        row['longitude_birth'] = longitude
    return row

merged_df = merged_df.apply(update_coordinates, axis=1)
merged_df['density'] = 1

# Initialisez l'application Dash
app = Dash(__name__, suppress_callback_exceptions=True)

# App layout
def Accueil_layout():
    return html.Div([
        html.Iframe(srcDoc=open('emmanuel.html',"r").read(), style={'border': 'none', 'width': '100%', 'height': '100vh'})
    ])

def page1_layout():
    return html.Div([
        html.Div(
            html.H1("Tableau",
                    style = {"font-family" : "verdana"}),
            style = { "background-color" : "antiquewhite"}),
        dash_table.DataTable(data=df.to_dict('records'), page_size=10),

        html.Div(
            html.H1("Tableau",
                    style = {"font-family" : "verdana"}),
            style = { "background-color" : "antiquewhite"}),
        dash_table.DataTable(data=merged_df.to_dict('records'), page_size=10),
        ])

In [14]:
country_coordinates

{'LA REUNION': ('-21.1151', '55.5364')}